In [3]:
'''Implementation of PCY Multihash'''
import pandas as pd
import itertools
import time

#2nd hashing func
def hash2(pairs, length):
    return int(pairs[0]) % length
    
#pass 1 - iterate through basket array, count quantity for each item and store in dict, create hashmap
def pass1(basketArray):
    itemCount = {}
    hashTable = {}
    hashTable2 = {}
    
    for i in range(len(basketArray)):
        lines = str(basketArray[i]).strip(' \n').split(' ') #string of items in basket
        for j in lines:
            if j in itemCount:
                itemCount[j] += 1
            else:
                itemCount[j] = 1

        #hash pair to bucket
        hashPairs = list(itertools.combinations(lines, 2)) 
        for k in range(len(hashPairs)):
            
            #hash fnc 1
            hashed1 = hash(hashPairs[k])
            #hash fnc 2
            hashed2 = hash2(hashPairs[k], len(basketArray))
            
            if hashed1 in hashTable:
                hashTable[hashed1] += 1  
            else:
                hashTable[hashed1] = 1

            if hashed2 in hashTable2:
                hashTable2[hashed2] += 1
            else:
                hashTable2[hashed2] = 1

    return itemCount, hashTable, hashTable2
    

#returns bitmap of buckets
def bitmap(hashTable, support):
    for i in hashTable.keys():
        if hashTable[i] > support:
            hashTable[i] = 1
        else:
            hashTable[i] = 0
    return hashTable
    

#check if each item in list is frequent
def checkFrequent(countList, support):
    frequentList = []
    for i in countList.keys():
        if countList[i] >= support:
            frequentList.append(i)
    return set(frequentList)
    

#pass 2 - find pairs where both elements are frequent 
def pass2(basketArray, frequentItems):
    candidateCount = {}

    for a in range(len(basketArray)):
        lines = str(basketArray[a]).strip(' \n').split(' ') #string of items in basket
        freqItems = [i for i in lines if i in frequentItems]
        pairsList = list(itertools.combinations (freqItems, 2)) #iterable list of pairs

        #compares pairs made from frequent items and pairs made from whole basket list
        for i in pairsList:
            if i in candidateCount:
                candidateCount[i] += 1
            else:
                candidateCount[i] = 1         
    return candidateCount
    

def multiHash(fileName, supp): 
    #read in file
    baskets = pd.read_csv(fileName, sep = '\t', header = None) 
    basketArray = baskets.values.ravel()
    support = int(supp * len(baskets)) 

    #pass 1 counting pairs and getting hashtables
    itemCount, hashTable1, hashTable2 = pass1(basketArray)
 
    #intermediate step
    frequentItems = checkFrequent(itemCount, support)
    hashTable1 = bitmap(hashTable1, support)
    hashTable2 = bitmap(hashTable2, support)
    
    #if no items are above support threshold
    if len(frequentItems) == 0 or len(hashTable1) == 0  or len(hashTable2) == 0:
        print("No frequent items")
        return

    #pass 2
    candidateCount = pass2(basketArray, frequentItems)
    
    #getting frequent pairs candidate
    frequentPairs = list(checkFrequent(candidateCount, support))

    for b in range(len(frequentPairs)):
        tempHash = hash(frequentPairs[b])
        tempHash2 = hash2(frequentPairs[b], len(basketArray))
     
        if hashTable1[tempHash] == 0 or hashTable2[tempHash2] == 0:
            frequentPairs.remove(b)
        else: 
            pass

    #writing list to txt file
    with open('frequentPairsMultiHash.txt','w+') as f:
        f.write(" ".join(map(str, frequentPairs)))
    return frequentPairs


st = time.time()   
frequentPairs = multiHash("retail.txt", 0.01)
print("frequent pairs:", len(frequentPairs))
et = time.time()
execution = et - st
print(execution)

frequent pairs: 58
12.08279824256897
